# data preprocess and words cut

- #### 以PTT推噓文為例 做資料前處理

In [1]:
import pandas as pd
import re
import os
import jieba # 專門用來處理中文斷詞的系統
import jieba.posseg as pseg 
import pickle
import numpy as np

In [2]:
## Find the path of this file

# 如果檔案需從外部進入可以這樣用 (turn back to main directory)
# os.chdir("../")
# article = pd.read_csv('Part1/Data/article_practice.csv')

import os
os.chdir("../")
os.getcwd()

'C:\\Users\\user\\Desktop\\Resume\\V2.5\\v2-5-nlp\\Part1'

## data overview

In [3]:
# load data (Set Path)
article = pd.read_csv('Data/article_practice.csv')

In [4]:
# Check Data
article.head()

,id,title,content,date,push,boo
0,M.1502380736.A.AC4,[問卦] 韓瑜 跟 周子瑜 怎麼選?,韓瑜是協志的前妻 也是很正的女演員 周子瑜是TWICE團裡裡面的台灣人 也是很正 這兩個要當...,Thu Aug 10 23:58:53 2017,4,4
1,M.1502380714.A.3B4,[問卦] 逢甲碟仙說兩岸統一到底是真是假！？,dear all 逢甲碟仙發生在民國七十五年三月中的事情， 一堆大學生玩碟仙然後發bbs 成...,Thu Aug 10 23:58:30 2017,14,3
2,M.1502380707.A.6A6,[問卦] 風雲起山河動,晚上好啊 各位 預備唱 風雲起 山河動 為什麼國軍早上都要唱這個啊 有沒有相關的八卦？？ S...,Thu Aug 10 23:58:22 2017,13,0
3,M.1502380681.A.18F,[問卦] 男朋友不睡覺怎麼辦,明天要早起 可是他不睡覺 在旁邊看Youtube這樣眼睛會壞掉 有沒有方法可以讓他早點睡 掛...,Thu Aug 10 23:57:58 2017,1,4
4,M.1502380627.A.383,[問卦] 英雄聯盟是不是已經漸漸過氣了?,一段時間沒在注意LOL了 結果發現各大LOL討論區人數有在明顯下降的趨勢 這實在令人滿驚訝的...,Thu Aug 10 23:57:04 2017,10,1


In [5]:
article['content']

0         韓瑜是協志的前妻 也是很正的女演員 周子瑜是TWICE團裡裡面的台灣人 也是很正 這兩個要當...
1         dear all 逢甲碟仙發生在民國七十五年三月中的事情， 一堆大學生玩碟仙然後發bbs 成...
2         晚上好啊 各位 預備唱 風雲起 山河動 為什麼國軍早上都要唱這個啊 有沒有相關的八卦？？ S...
3         明天要早起 可是他不睡覺 在旁邊看Youtube這樣眼睛會壞掉 有沒有方法可以讓他早點睡 掛...
4         一段時間沒在注意LOL了 結果發現各大LOL討論區人數有在明顯下降的趨勢 這實在令人滿驚訝的...
                                ...                        
252231    看影片每個信徒一接近SEAFOOD還是雙手合十就哭得一把鼻涕一把眼淚的 讓小魯不禁想問 SE...
252232    諸位晚安 如題 不知道諸位比較喜歡吃seafood還是skyfood? 不才在下先拋磚引玉 ...
252233    utube上幾千人的場合 一起喊 感謝seafood 讚嘆seafood 一起跪拜 一起起乩...
252234              剛剛打死了連念都不會念的蚰蜒 有沒有他的八卦？ 我覺得他看起來比蟑螂好欺負說 
252235    小魯早上在往新部隊的軍車上哼歌 就有個穿便服的白目一直看著我笑 到部隊後 一下車值星就叫我們...
Name: content, Length: 252236, dtype: object

In [7]:
article['content'] = article['content'].str.replace('https?:\/\/\S*', '')
article['content']

# ex2: 將 ''(空值) 取代為'NaN'
article['content'] = article['content'].replace('', np.NaN)

article['idx'] = article.index
article['idx']

0         韓瑜是協志的前妻 也是很正的女演員 周子瑜是TWICE團裡裡面的台灣人 也是很正 這兩個要當...
1         dear all 逢甲碟仙發生在民國七十五年三月中的事情， 一堆大學生玩碟仙然後發bbs 成...
2         晚上好啊 各位 預備唱 風雲起 山河動 為什麼國軍早上都要唱這個啊 有沒有相關的八卦？？ S...
3         明天要早起 可是他不睡覺 在旁邊看Youtube這樣眼睛會壞掉 有沒有方法可以讓他早點睡 掛...
4         一段時間沒在注意LOL了 結果發現各大LOL討論區人數有在明顯下降的趨勢 這實在令人滿驚訝的...
                                ...                        
252231    看影片每個信徒一接近SEAFOOD還是雙手合十就哭得一把鼻涕一把眼淚的 讓小魯不禁想問 SE...
252232    諸位晚安 如題 不知道諸位比較喜歡吃seafood還是skyfood? 不才在下先拋磚引玉 ...
252233    utube上幾千人的場合 一起喊 感謝seafood 讚嘆seafood 一起跪拜 一起起乩...
252234              剛剛打死了連念都不會念的蚰蜒 有沒有他的八卦？ 我覺得他看起來比蟑螂好欺負說 
252235    小魯早上在往新部隊的軍車上哼歌 就有個穿便服的白目一直看著我笑 到部隊後 一下車值星就叫我們...
Name: content, Length: 252236, dtype: object

In [8]:
    # check type
type(article['content'])

# 1.Preprocessing  :  filter rules(依據Domain Knowledge去除不需要的資訊)
   
    # 1.1 ex1: 將'https?:\/\/\S*' 取代為' '(空值)
article['content'] = article['content'].str.replace('https?:\/\/\S*', '')

    # 1.2 將 ''(空值) 取代為'NaN'
article['content'] = article['content'].replace('', np.NaN)

article['idx'] = article.index
article['idx']

    # 1.3 
article = article.dropna()
article

,id,title,content,date,push,boo,idx
0,M.1502380736.A.AC4,[問卦] 韓瑜 跟 周子瑜 怎麼選?,韓瑜是協志的前妻 也是很正的女演員 周子瑜是TWICE團裡裡面的台灣人 也是很正 這兩個要當...,Thu Aug 10 23:58:53 2017,4,4,0
1,M.1502380714.A.3B4,[問卦] 逢甲碟仙說兩岸統一到底是真是假！？,dear all 逢甲碟仙發生在民國七十五年三月中的事情， 一堆大學生玩碟仙然後發bbs 成...,Thu Aug 10 23:58:30 2017,14,3,1
2,M.1502380707.A.6A6,[問卦] 風雲起山河動,晚上好啊 各位 預備唱 風雲起 山河動 為什麼國軍早上都要唱這個啊 有沒有相關的八卦？？ S...,Thu Aug 10 23:58:22 2017,13,0,2
3,M.1502380681.A.18F,[問卦] 男朋友不睡覺怎麼辦,明天要早起 可是他不睡覺 在旁邊看Youtube這樣眼睛會壞掉 有沒有方法可以讓他早點睡 掛...,Thu Aug 10 23:57:58 2017,1,4,3
4,M.1502380627.A.383,[問卦] 英雄聯盟是不是已經漸漸過氣了?,一段時間沒在注意LOL了 結果發現各大LOL討論區人數有在明顯下降的趨勢 這實在令人滿驚訝的...,Thu Aug 10 23:57:04 2017,10,1,4
...,...,...,...,...,...,...,...
252231,M.1505405210.A.C69,[問卦] 信徒能感覺到SEAFOOD的靈壓嗎,看影片每個信徒一接近SEAFOOD還是雙手合十就哭得一把鼻涕一把眼淚的 讓小魯不禁想問 SE...,Fri Sep 15 00:06:47 2017,3,0,252231
252232,M.1505404956.A.DDC,[問卦] 比較喜歡吃seafood還是skyfood?,諸位晚安 如題 不知道諸位比較喜歡吃seafood還是skyfood? 不才在下先拋磚引玉 ...,Fri Sep 15 00:02:30 2017,1,1,252232
252233,M.1505404929.A.7F0,[問卦] seafood幾千人的場合真的沒有一個臥底,utube上幾千人的場合 一起喊 感謝seafood 讚嘆seafood 一起跪拜 一起起乩...,Fri Sep 15 00:02:04 2017,4,0,252233
252234,M.1505404909.A.4CF,[問卦] 有沒有蚰蜒的八卦,剛剛打死了連念都不會念的蚰蜒 有沒有他的八卦？ 我覺得他看起來比蟑螂好欺負說,Fri Sep 15 00:01:46 2017,4,3,252234


In [8]:
article['content']

0         韓瑜是協志的前妻 也是很正的女演員 周子瑜是TWICE團裡裡面的台灣人 也是很正 這兩個要當...
1         dear all 逢甲碟仙發生在民國七十五年三月中的事情， 一堆大學生玩碟仙然後發bbs 成...
2         晚上好啊 各位 預備唱 風雲起 山河動 為什麼國軍早上都要唱這個啊 有沒有相關的八卦？？ S...
3         明天要早起 可是他不睡覺 在旁邊看Youtube這樣眼睛會壞掉 有沒有方法可以讓他早點睡 掛...
4         一段時間沒在注意LOL了 結果發現各大LOL討論區人數有在明顯下降的趨勢 這實在令人滿驚訝的...
                                ...                        
252231    看影片每個信徒一接近SEAFOOD還是雙手合十就哭得一把鼻涕一把眼淚的 讓小魯不禁想問 SE...
252232    諸位晚安 如題 不知道諸位比較喜歡吃seafood還是skyfood? 不才在下先拋磚引玉 ...
252233    utube上幾千人的場合 一起喊 感謝seafood 讚嘆seafood 一起跪拜 一起起乩...
252234              剛剛打死了連念都不會念的蚰蜒 有沒有他的八卦？ 我覺得他看起來比蟑螂好欺負說 
252235    小魯早上在往新部隊的軍車上哼歌 就有個穿便服的白目一直看著我笑 到部隊後 一下車值星就叫我們...
Name: content, Length: 252229, dtype: object

In [9]:
article['idx']

0              0
1              1
2              2
3              3
4              4
           ...  
252231    252231
252232    252232
252233    252233
252234    252234
252235    252235
Name: idx, Length: 252229, dtype: int64

In [10]:
# remove data

#remove NaN
article = article.dropna()

#讓index重置成原本的樣子
article = article.reset_index(drop=True)

article['idx'] = article.index
article['idx']


# String Split 
# ex: 依照文字之間的空格split

article['content'].str.split( " ", n=4, expand = True )

,0,1,2,3,4
0,韓瑜是協志的前妻,也是很正的女演員,周子瑜是TWICE團裡裡面的台灣人,也是很正,這兩個要當鄉民的老婆，你要怎麼選呢?? 五樓你真有勇氣
1,dear,all,逢甲碟仙發生在民國七十五年三月中的事情，,一堆大學生玩碟仙然後發bbs,成功預測921大地震，小弟預言都還沒出生呢。 後面說預言一百一十六年兩岸統一， 是我們統一對...
2,晚上好啊,各位,預備唱,風雲起,山河動 為什麼國軍早上都要唱這個啊 有沒有相關的八卦？？ Sent from JPTT on...
3,明天要早起,可是他不睡覺,在旁邊看Youtube這樣眼睛會壞掉,有沒有方法可以讓他早點睡,掛？ Sent from JPTT on my HTC_D10i.
4,一段時間沒在注意LOL了,結果發現各大LOL討論區人數有在明顯下降的趨勢,這實在令人滿驚訝的,曾經一時的遊戲霸主,難道也要漸漸過氣了嗎 不過LOL確實也撐很久了 能到現在也算厲害了 只是遊戲玩久了還是會膩吧?
...,...,...,...,...,...
252224,看影片每個信徒一接近SEAFOOD還是雙手合十就哭得一把鼻涕一把眼淚的,讓小魯不禁想問,SEAFOOD是不是有特殊的靈壓才能讓信徒感受到,,None
252225,諸位晚安,如題,不知道諸位比較喜歡吃seafood還是skyfood?,不才在下先拋磚引玉,在下是都喜歡 不管是魚肉亦或是雞肉在下覺得都不錯 不知道諸位意下如何? 卦否? 風清揚微笑道...
252226,utube上幾千人的場合,一起喊,感謝seafood,讚嘆seafood,一起跪拜 一起起乩 動作整齊化一 就像同一人 這怎麼辦到的阿 難道裡面沒有半個臥底？？？ 尤...
252227,剛剛打死了連念都不會念的蚰蜒,有沒有他的八卦？,我覺得他看起來比蟑螂好欺負說,,None


In [7]:
article.to_csv('data/article_preprocessed.csv', index=False)

# jieba

## cut word : 自然語言處理第一招 : 處理斷詞 

- ### 斷詞是甚麼? 中文詞句的斷句!

In [8]:
## set dictionary (can define yourself)

# jieba段詞的dictionary(有很多種 這邊只是某個範例)
jieba.set_dictionary('jieba/dict.txt.big')

#斷詞檔案 :  jieba為中文斷詞的資料庫 
stop_words = open('jieba/stop_words.txt',encoding="utf-8").read().splitlines()

In [9]:
print(stop_words[:300])

['\ufeff,', '?', '、', '。', '“', '”', '《', '》', '！', '，', '：', '；', '？', '人民', '末##末', '啊', '阿', '哎', '哎呀', '哎喲', '唉', '我', '我們', '按', '按照', '依照', '吧', '吧噠', '把', '罷了', '被', '本', '本著', '比', '比方', '比如', '鄙人', '彼', '彼此', '邊', '別', '別的', '別說', '並', '並且', '不比', '不成', '不單', '不但', '不獨', '不管', '不光', '不過', '不僅', '不拘', '不論', '不怕', '不然', '不如', '不特', '不惟', '不問', '不只', '朝', '朝著', '趁', '趁著', '乘', '沖', '除', '除此之外', '除非', '除了', '此', '此間', '此外', '從', '從而', '打', '待', '但', '但是', '當', '當著', '到', '得', '的', '的話', '等', '等等', '地', '第', '叮咚', '對', '對於', '多', '多少', '而', '而況', '而且', '而是', '而外', '而言', '而已', '爾後', '反過來', '反過來說', '反之', '非但', '非徒', '否則', '嘎', '嘎登', '該', '趕', '個', '各', '各個', '各位', '各種', '各自', '給', '根據', '跟', '故', '故此', '固然', '關於', '管', '歸', '果然', '果真', '過', '哈', '哈哈', '呵', '和', '何', '何處', '何況', '何時', '嘿', '哼', '哼唷', '呼哧', '乎', '嘩', '還是', '還有', '換句話說', '換言之', '或', '或是', '或者', '極了', '及', '及其', '及至', '即', '即便', '即或', '即令', '即若', '即使', '幾', '幾時', '己', '既', '既然', '既是', '繼而', '加之', '假如', '假若', '假使', '鑒於', 

In [10]:
data = pd.read_csv('data/article_preprocessed.csv')
data = data['content'].tolist()

In [11]:
print(data[:5])

['韓瑜是協志的前妻 也是很正的女演員 周子瑜是TWICE團裡裡面的台灣人 也是很正 這兩個要當鄉民的老婆，你要怎麼選呢?? 五樓你真有勇氣', 'dear all 逢甲碟仙發生在民國七十五年三月中的事情， 一堆大學生玩碟仙然後發bbs 成功預測921大地震，小弟預言都還沒出生呢。 後面說預言一百一十六年兩岸統一， 是我們統一對岸，還是對岸統一我們...... 應該是不用猜了。 只是真的存在預言這種事情嗎？ 倒底會不會在116被統？ 我知道資料庫發文日期可以被輕鬆改變， 沒有拍照的狀況下...... 碟仙到底是真的假的？ 到底有沒有科學或是經驗法則來破解， 這到底是謠言還是真實的八卦？ ', '晚上好啊 各位 預備唱 風雲起 山河動 為什麼國軍早上都要唱這個啊 有沒有相關的八卦？？ Sent from JPTT on my Xiaomi Redmi Note 4. ', '明天要早起 可是他不睡覺 在旁邊看Youtube這樣眼睛會壞掉 有沒有方法可以讓他早點睡 掛？ Sent from JPTT on my HTC_D10i. ', '一段時間沒在注意LOL了 結果發現各大LOL討論區人數有在明顯下降的趨勢 這實在令人滿驚訝的 曾經一時的遊戲霸主 難道也要漸漸過氣了嗎 不過LOL確實也撐很久了 能到現在也算厲害了 只是遊戲玩久了還是會膩吧? ']


In [12]:
sentences = []

for i, text in enumerate(data):
    line = []

    for w in jieba.cut(text, cut_all=False):
        
        ## remove stopwords and digits
        ## can define your own rules
        if w not in stop_words and not bool(re.match('[0-9]+', w)):
            line.append(w)

    sentences.append(line)

    if i%10000==0:
        print(i, '/', len(data))

Building prefix dict from C:\Users\user\Desktop\Resume\V2.5\v2-5-nlp\Part1\jieba\dict.txt.big ...
Dumping model to file cache C:\Users\user\AppData\Local\Temp\jieba.ucad950f0f18150a2aa2bae84a4bcd52a.cache
Loading model cost 1.598 seconds.
Prefix dict has been built successfully.


0 / 252229
10000 / 252229
20000 / 252229
30000 / 252229
40000 / 252229
50000 / 252229
60000 / 252229
70000 / 252229
80000 / 252229
90000 / 252229
100000 / 252229
110000 / 252229
120000 / 252229
130000 / 252229
140000 / 252229
150000 / 252229
160000 / 252229
170000 / 252229
180000 / 252229
190000 / 252229
200000 / 252229
210000 / 252229
220000 / 252229
230000 / 252229
240000 / 252229
250000 / 252229


In [13]:
print(sentences[0:20])

[['韓瑜', '協志', '前妻', '正', '女演員', '周子', '瑜', 'TWICE', '團裡裡面', '台灣', '人', '正', '兩個', '要當', '鄉民', '老婆', '選', '五樓', '真', '勇氣'], ['dear', 'all', '逢甲', '碟仙', '發生', '民國', '七十五年', '三月中', '事情', '一堆', '大學生', '玩', '碟仙', '後發', 'bbs', '成功', '預測', '地震', '小弟', '預言', '都還沒', '出生', '後面', '說', '預言', '一百', '一十六年', '兩岸', '統一', '統一', '對岸', '對岸', '統一', '應該', '不用', '猜', '真的', '存在', '預言', '這種', '事情', '倒底', '被統', '知道', '資料庫', '發文', '日期', '輕鬆', '改變', '拍照', '狀況', '下', '碟仙', '真的假', '有沒有', '科學', '經驗', '法則', '破解', '謠言', '真實', '八卦'], ['晚上', '好', '預備', '唱', '風雲', '山河', '動', '國軍', '早上', '唱', '有沒有', '相關', '八卦', 'Sent', 'from', 'JPTT', 'on', 'my', 'Xiaomi', 'Redmi', 'Note'], ['明天', '早起', '睡覺', '旁邊', 'Youtube', '眼睛', '壞掉', '有沒有', '方法', '早點', '睡', '掛', 'Sent', 'from', 'JPTT', 'on', 'my', 'HTC', 'D10i'], ['一段時間', '注意', 'LOL', '發現', '各大', 'LOL', '討論區', '人數', '明顯', '下降', '趨勢', '實在', '令人', '驚訝', '曾經', '一時', '遊戲', '霸主', '漸漸', '過氣', 'LOL', '確實', '撐', '久', '現在', '算', '厲害', '遊戲', '玩久', '會膩'], ['總覺', '每天', '睡', '睡覺時間', '睡', '不著', '長

In [14]:
## save data as pickle format
with open("data/article_cutted", "wb") as file : 
    pickle.dump(sentences, file)

- ## posseg (詞性)

 #### 在斷詞時 , 請用 jieba 把詞性抓出來 (1 筆即可)

In [15]:
import jieba.posseg as pseg

In [16]:
for w, f in pseg.cut(data[0]):
    print(w, ' ', f)

韓瑜   nr
是   v
協志   n
的   uj
前妻   n
    x
也   d
是   v
很正   a
的   uj
女演員   x
    x
周子瑜   nr
是   v
TWICE   eng
團裡   q
裡面   f
的   uj
台灣   ns
人   n
    x
也   d
是   v
很正   d
    x
這   zg
兩個   x
要   v
當   p
鄉民   x
的   uj
老婆   n
，   x
你   r
要   v
怎麼   x
選   v
呢   y
?   x
?   x
    x
五   m
樓   n
你   r
真   d
有   v
勇氣   x
